# Import Libraries

In [12]:
from cm_api import get_api_token
from data_agg import *
from cm_config import  config
from helper_funct1 import generate_yesterday_date
from data_agg import collect_artist_sppop
from data_agg import create_artist_ig_engagement_feat
import datetime
import pandas as pd
from graph_maker import make_barplot

In [13]:
rt = config['refresh_token']
api_token = get_api_token(rt)

# Collect Data From Tiktok Weekly Chart

In [ ]:
#get yesterday's date
yesterday_date = generate_yesterday_date()

#get tiktok chart
data = get_tiktok_chart_data(api_token, 'tracks', yesterday_date, 'weekly')

# Parse Tiktok Data
parse raw data into pandas dataframe with the following columns:

'rank', 'added_at', 'title', 'artist', 'isrc', 'velocity', 'cm_id', 'time_on_chart', 'release_dates']

In [ ]:
#parse tiktok data into dataframe
dataframe = parse_tiktok_data(data)

# Get Artist IDs (Chartmetric)

In [ ]:
#get chartmetric id and add to dataframe
dataframe = create_artist_id_feat(api_token, dataframe)

# Data Cleaning

## remove timestamp from add date

In [ ]:
#remove timestamp from `added_at` feature
dataframe['added_at'] = pd.to_datetime(dataframe['added_at']).dt.date

# Feature Engineer - Track Age
categorial value for track age on chart
`a month` == x <=30; `2 months` 60 >= x > 30; `over 2 months` x> 60

In [ ]:
dataframe['track_age'] = dataframe['time_on_chart'].apply(lambda x: 'a month' if int(x) <= 30 else ('2 months' if 60 >= int(x) > 30 else 'over 2 months'))

## Filter dataset down to records with track age of a month

In [ ]:
#filter dataset down to 1-month old tracks
young_trax = dataframe.loc[dataframe['track_age'] == 'a month'].reset_index().drop(axis=1, labels='index')

## Engineer `Before_Tiktok_Date`
7 days before record's first occurance on Tiktok chart. `added_at` - (`time_on_chart` + 7)

In [ ]:
#engineer date feature for 7 days before each record's first occurance on the chart
young_trax = create_before_tiktok_date_feat(young_trax)
    

# Find True Artist Names

In [ ]:
#replace artist names with real artist names
young_trax = collect_true_artist_names(api_token, young_trax)

# Find IG followers for `Before_Tiktok_Date`

In [ ]:

#collect instagram followers for `Before_Tiktok_Date`
young_trax = create_instagram_followers_before_feat(api_token, young_trax)

# Collect Current Spotify Popularity Metric

In [ ]:
#Collect currect spotify popularity metric for each artist
young_trax = collect_artist_sppop(api_token, young_trax)

# Create Spotify Popularity Index

In [ ]:
#Career Stage feature
young_trax['Popularity-Index'] = young_trax['artist_pop'].apply(lambda x: '>= 90' if x >= 90 else ('>=80' if x >= 80 else '< 80'))

# Date 7 days after record's first chart appearance
subtract `time_on_chart` from `added_at`

In [ ]:
#adding date for 7 days after record's first chart appearance
young_trax['Week_In_Chart_Date'] = young_trax['Before_Tiktok_Date'] + datetime.timedelta(days=14)

# Collect Total IG Followers 1 week after chart start date

In [ ]:
young_trax = create_instagram_followers_after_feat(api_token, young_trax)

# Percent IG Follower Gain
difference between a week before and a week after being on the chart

In [ ]:
#percent gain in IG followers
young_trax['IG_Follower_Gain-%'] = (young_trax['ig_followers_afters'] - young_trax['total_ig_followers'])/young_trax['total_ig_followers']*100

# Collect Instagram E

In [ ]:
#adding instagram engagement before tiktok chart
young_trax = create_artist_ig_engagement_feat(api_token, young_trax)

# Engineer `total_spotify_followers`
total Spotify followers on `Before_Tiktok_Date`

In [ ]:
def create_spotify_followers_before_feat(dataframe):
    from cm_api import get_fan_metrics
    bucket = []
    for row in dataframe.iterrows():
        before_date = row[1]['Before_Tiktok_Date']
        after_date = row[1]['Before_Tiktok_Date'] + datetime.timedelta(days=5)
        artist_id = row[1]['cm_artist_id']
        data = get_fan_metrics(api_token, artist_id,'spotify', before_date, after_date, field='followers')['followers']
        if len(data) > 0:
            tup = (data[0]['value'], data[-1]['value'])
            bucket.append(tup)
        else:
            tup = (None, None)
            bucket.append(tup) 

    dataframe['total_spotify_followers'] = [x[0] for x in bucket]
    return dataframe


In [ ]:
df2 = create_spotify_followers_before_feat(df1)

In [ ]:
young_trax = all_trax.loc[all_trax['track_age'] == 'a month'].reset_index().drop(axis=1, labels='index')


In [ ]:
def create_instagram_followers_before_feat(dataframe):
    from cm_api import get_fan_metrics
    bucket = []
    for row in dataframe.iterrows():
        before_date = str(row[1]['Before_Tiktok_Date'])[:10]
        after_date = str(row[1]['Before_Tiktok_Date'] + datetime.timedelta(days=5))[:10]
        artist_id = row[1]['cm_artist_id']
        data = get_fan_metrics(api_token, artist_id,'instagram', str(before_date), str(after_date), field='followers')['followers']
        if len(data) > 0:
            tup = (data[0]['value'], data[-1]['value'])
            bucket.append(tup)
        else:
            tup = (None, None)
            bucket.append(tup) 

    dataframe['total_ig_followers'] = [x[0] for x in bucket]
    return dataframe

In [ ]:
young_trax = create_instagram_followers_before_feat(young_trax)

## Find True Artist Names

In [ ]:
def collect_true_artist_names(dataframe):
    from cm_api import get_track_metadata
    bucket = []
    for row in dataframe.iterrows():
        track_id = row[1]['cm_id']
        artist_name = get_track_metadata(api_token, track_id)['artists'][0]['name']
        bucket.append(artist_name)

    dataframe['artist'] = bucket

    return dataframe
    


In [ ]:
all_trax = collect_true_artist_names(all_trax)

# Collect Popularity metric for each artist

In [ ]:
young_trax = collect_artist_sppop(api_token, new_trax)

# Career Stage
`Established` if spotify popularity >= 88; `Emerging` if spotify popularity >= 80 else indie

In [ ]:
new_trax['Popularity-Index'] = new_trax['artist_pop'].apply(lambda x: '>= 90' if x >= 90 else ('>=80' if x >= 80 else '< 80'))


In [ ]:
new_trax['Week_In_Chart_Date'] = new_trax['Before_Tiktok_Date'] + datetime.timedelta(days=14)

# Instagram Gain
percent gain from `Before_Tiktok_Date` to `Week_In_Chart_Date`

## Collect IG Followers for `Week_In_Chart_Date`

In [ ]:
new_trax = create_instagram_followers_after_feat(new_trax)

In [ ]:
new_trax['IG_Follower_Gain-%'] = (new_trax['ig_followers_afters'] - new_trax['total_ig_followers'])/new_trax['total_ig_followers']*100

# Instagram Engagement (Before Tiktok)

In [ ]:
new_trax.head()

In [ ]:
#given artist id and date, this function iterates thru a dataframe and 
#creates new feature for instagram engagement
def create_artist_ig_engagement_feat(api_token, dataframe):
    from cm_api import get_instagram_audience
    bucket = []
    for row in dataframe.iterrows():
        artist_id = row[1]['cm_artist_id']
        date = str(row[1]['Before_Tiktok_Date'])[:10]

        ig_data = get_instagram_audience(api_token, artist_id, date)
        if len(ig_data) > 0:
            eng_rate = ig_data.get('engagement_rate', 0)
            bucket.append(eng_rate)
        else:
            bucket.append(None)

    dataframe['ig_eng_before'] = bucket
    return dataframe


In [ ]:
new_trax = create_artist_ig_engagement_feat(api_token, new_trax)

In [32]:
get_instagram_audience(api_token, 1114822,)

{'top_countries': [{'name': 'United States',
   'code': 'US',
   'percent': '94.04',
   'followers': 343326},
  {'name': 'Canada', 'code': 'CA', 'percent': '0.51', 'followers': 1845},
  {'name': 'France', 'code': 'FR', 'percent': '0.40', 'followers': 1444},
  {'name': 'The Bahamas', 'code': 'BS', 'percent': '0.35', 'followers': 1283},
  {'name': 'United Kingdom',
   'code': 'GB',
   'percent': '0.31',
   'followers': 1123},
  {'name': 'Dominican Republic',
   'code': 'DO',
   'percent': '0.29',
   'followers': 1043},
  {'name': 'India', 'code': 'IN', 'percent': '0.24', 'followers': 882}],
 'top_cities': [{'name': 'Orlando',
   'percent': '7.08',
   'followers': 25835,
   'state': 'Florida',
   'country': 'United States'},
  {'name': 'Jacksonville',
   'percent': '4.24',
   'followers': 15485,
   'state': 'Florida',
   'country': 'United States'},
  {'name': 'Tampa',
   'percent': '3.10',
   'followers': 11313,
   'state': 'Florida',
   'country': 'United States'},
  {'name': 'Tallahass

# Save Dataset



In [ ]:
#save full dataset - 85 songs
all_trax.to_csv('/home/bull/Documents/bull-analytics/blog7/data/top_tiktok_artists.csv', index_label='index')

In [ ]:
#save dataset of 1 month old records
new_trax.to_csv('/home/bull/Documents/bull-analytics/blog7/data/one_month_trax.csv', index_label='index')

# Read in Datasets

In [ ]:
all_trax = pd.read_csv('/home/bull/Documents/bull-analytics/blog7/data/top_tiktok_artists.csv', index_col=0,
                      parse_dates=['added_at','Before_Tiktok_Date'])

In [6]:
new_trax = pd.read_csv('/home/bull/Documents/bull-analytics/blog7/data/one_month_trax.csv', index_col=0,
                      parse_dates=['added_at','Before_Tiktok_Date'])

# Summary Statistics for playlist tracks

In [ ]:
summ_stat_playlist_artist_popularity(api_token, playlist_id)

Median popularity for top hit artists is 88

# Pop Rising popularity of artists

In [ ]:
data = get_playlist_tracks(api_token, 'spotify', 984513, 'current', 'us')
bucket = []
for track in data:
    tup = (track['cm_artist'][0])
    bucket.append(tup)
df = pd.DataFrame(bucket, columns=['cm_artist_id'])

bucket = []
for row in df.iterrows():
    x = row[1]['cm_artist_id']
    artist_pop = get_artist_metadata(api_token, x)['cm_statistics']['sp_popularity']
    print
    if (artist_pop):
        bucket.append(artist_pop)
    else:
        bucket.append(None)

df['artist_pop'] = bucket

get_summary_statistics(df['artist_pop'].dropna())

Rising artists median popularity is 80

# Analysis

In [ ]:
def make_countplot(dataframe,x_series):

    fig = figure(figsize=(20,12))  # define the figure window
    ax1  = fig.add_subplot(111)   # define the axis
    graph = sns.countplot(data=dataframe, x=x_series)
    plt.xticks(rotation=0)
    plt.title('Top Trending Tiktok Tracks - Week of March 9, 2021', size=25)
    plt.ylabel('# of Artists', size=25)
    plt.xlabel('Track Age', size=25)
    plt.ticklabel_format(style='plain', axis='y') #change xaxis from sci to plain style
    plt.tick_params(axis='both', which='major', labelsize=20)

    for bar in graph.patches:
        graph.annotate((format((bar.get_height()/len(df2))*100, '.2f'))+'%', (bar.get_x() + bar.get_width()/2, bar.get_height()+.5), size=20)

    plt.tight_layout()
#     plt.savefig('/home/bull/Documents/bull-analytics/blog7/visuals/track_age.jpeg')
    plt.show()

## Top New Artists by Instagram Followers

In [ ]:
# sorted_df.head()

In [ ]:
def make_barplot(dataframe, x_series, y_series):

    import matplotlib.pyplot as plt 
    from helper_funct import insert_thousands_commas
    fig = figure(figsize=(35,25))  # define the figure window
    ax1  = fig.add_subplot(111)   # define the axis
    sorted_df = dataframe.sort_values(y_series, ascending=False)[:10]
    graph = sns.barplot(data=sorted_df, x=x_series, y=y_series)
    #label x,y axes
    plt.title('Newest Artists Trending on Tiktok (1-Month on Chart)', size=40)
    plt.xticks(rotation=90)
    plt.ylabel('Total IG Followers - Before Tiktok', size=40)
    plt.xlabel('Artists', size=40)
    plt.ticklabel_format(style='plain', axis='y') #change xaxis from sci to plain style
    plt.tick_params(axis='both', which='major', labelsize=35)
    plt.grid(axis='y', which='major')
    #annotate bars
    oldheight=0
    for bar in graph.patches:
        currentheight = bar.get_height()
        if currentheight > oldheight:

            graph.annotate(insert_thousands_commas(bar.get_height()), (bar.get_x() + bar.get_width()/2, bar.get_height()), 
                           size=40, xytext=(bar.get_x() + bar.get_width()/10, currentheight+3000000), 
                           arrowprops={'arrowstyle':'wedge'})
            oldheight = bar.get_height()
        else:
            graph.annotate(insert_thousands_commas(bar.get_height()), (bar.get_x() + bar.get_width()/2, bar.get_height()), 
                           size=40,xytext=(bar.get_x() + bar.get_width()/10, currentheight+oldheight/2), 
                           arrowprops={'arrowstyle':'wedge'})
            oldheight = bar.get_height()     

    plt.tight_layout()
#     plt.savefig('/home/bull/Documents/bull-analytics/blog7/visuals/top10_artistsIG.jpeg')
    plt.show()

In [ ]:
make_barplot(new_trax, 'artist', 'total_ig_followers')

In [ ]:
from matplotlib.pyplot import * 

fig = figure(figsize=(30,15))  # define the figure window
ax1  = fig.add_subplot(111)   # define the axis
sns.boxplot(new_trax['total_ig_followers'], fliersize=1,linewidth=2, whis=30,width=.75, color='yellow')
#markers with just twitter
sns.swarmplot(data=new_trax,x=new_trax['total_ig_followers'], color='green', size=25)
plt.ticklabel_format(style='plain', axis='x')
plt.xlabel('Total IG Followers - Before Tiktok', size=25)
plt.xticks(size=20)
plt.show()



In [ ]:
get_summary_statistics(yt2['total_ig_followers'].dropna())

In [ ]:
#filter dataset to top 50% of total spotify followers

top_artists = yt2.loc[yt2['total_ig_followers'] >= yt2['total_ig_followers'].quantile(.5)]

In [ ]:
top_artists.sort_values('total_spotify_followers', ascending=False, inplace=True)

In [ ]:
fig = figure(figsize=(30,15))  # define the figure window
ax1  = fig.add_subplot(111)   # define the axis
sns.barplot(data=top_artists, x='artist', y='total_spotify_followers')
plt.xticks(rotation=45)
plt.ticklabel_format(style='plain', axis='y') #change xaxis from sci to plain style
plt.tick_params(axis='both', which='major', labelsize=25)


plt.show()